# Call center case - Post-call transcription and analysis with Azure OpenAI Service

In this exercise, we will perform sentiment analysis and summerization using call center transcriptions. We will transribe the customer recording to text, then use OpenAI to detect sentiment. We also use OpenAI to summerize long text into a few sentences for further analysis.

In [15]:
from dotenv import load_dotenv
from pathlib import Path
env_path = Path('C:\Labfiles\call_center\.env') # Change with your .env file
load_dotenv(dotenv_path=env_path,override=True)

True

In [10]:
# from azure.ai.textanalytics import TextAnalyticsClient
# from azure.core.credentials import AzureKeyCredential
import azure.cognitiveservices.speech as speechsdk

import json, os
import string
import time
import wave

import openai
import re
import requests
import sys
from num2words import num2words
import os
import pandas as pd
import numpy as np
from openai.embeddings_utils import get_embedding, cosine_similarity
from transformers import GPT2TokenizerFast

openai.api_type = "azure"
openai.api_key = os.getenv('OPENAI_API_KEY') 
openai.api_base = os.getenv('OPENAI_API_BASE') 
openai.api_version = "2023-05-15"

SPEECH_KEY = os.environ["SPEECH_API_KEY"]

COMPLETIONS_MODEL = os.environ["COMPLETIONS_MODEL"]

def recognize_speech_from_file(filename):
    # Set up the subscription info for the Speech Service:
    # Replace with your own subscription key and service region (e.g., "westus").
    speech_key = SPEECH_KEY
    service_region = "eastus"

    speech_config = speechsdk.SpeechConfig(subscription=speech_key, region=service_region)
    audio_config = speechsdk.audio.AudioConfig(filename=filename)
    # Creates a speech recognizer using a file as audio input, also specify the speech language
    speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config,  audio_config=audio_config)

    global done 
    done = False
    global recognized_text_list 
    recognized_text_list=[]
    def stop_cb(evt: speechsdk.SessionEventArgs):
        """callback that signals to stop continuous recognition upon receiving an event `evt`"""
        print('CLOSING on {}'.format(evt))
        global done
        done = True

    def recognize_cb(evt: speechsdk.SpeechRecognitionEventArgs):
        """callback for recognizing the recognized text"""
        global recognized_text_list
        recognized_text_list.append(evt.result.text)
        # print('RECOGNIZED: {}'.format(evt.result.text))

    # Connect callbacks to the events fired by the speech recognizer
    # speech_recognizer.recognizing.connect(lambda evt: print('RECOGNIZING: {}'.format(evt)))
    speech_recognizer.recognized.connect(recognize_cb)
    speech_recognizer.session_started.connect(lambda evt: print('STT SESSION STARTED: {}'.format(evt)))
    speech_recognizer.session_stopped.connect(lambda evt: print('STT SESSION STOPPED {}'.format(evt)))
    # speech_recognizer.canceled.connect(lambda evt: print('CANCELED {}'.format(evt)))
    # stop continuous recognition on either session stopped or canceled events
    speech_recognizer.session_stopped.connect(stop_cb)
    # speech_recognizer.canceled.connect(stop_cb)

    # Start continuous speech recognition
    speech_recognizer.start_continuous_recognition()
    while not done:
        time.sleep(.5)

    speech_recognizer.stop_continuous_recognition()

    return recognized_text_list

# Sentiment Analysis
### Transcribe Customer Call to Text

In [11]:
text = recognize_speech_from_file("../data/good_review.wav")
print(text)

STT SESSION STARTED: SessionEventArgs(session_id=ea5f9ba54b9246c8893dcbeb0c9d305e)
STT SESSION STOPPED SessionEventArgs(session_id=ea5f9ba54b9246c8893dcbeb0c9d305e)
CLOSING on SessionEventArgs(session_id=ea5f9ba54b9246c8893dcbeb0c9d305e)
['I had purchased another thermostat from a big box store and the Rep there assured me that it would turn off when it got too cold.', 'Well, come to find out, it did not and my heater in the garage was running a lot of the time.', 'This one was great and was easily wired.', 'It does turn off when you turn it all the way to the left.', "It's annoying they don't have any in Celsius, but whatever.", 'It runs well and actually turns off my garage heater.', 'Would totally purchase again.', 'Installed with five kilowatts heater in garage.', 'Thermostat failed an on position overnight and came close to fire with overheated connections. See photos.', 'Installation was by certified electrician.']


### Create Promot for sentiment analysis
Use natural language to instruct OpenAI to detect customer's sentiment

In [12]:
prompt = f"Detect whether customer is positive or negative.  Just say positive or negative.\n\n{' '.join(text)}"

openai.Completion.create(
    prompt=prompt,
    temperature=0,
    max_tokens=300,
    engine=COMPLETIONS_MODEL
)["choices"][0]["text"].strip(" \n")

'Thermostat was only in use for two weeks. I would not recommend this product. I have a 240v 30amp garage heater. I installed this thermostat and it worked great for about 2 weeks. Then it failed in the on position and the heater ran all night. The connections overheated and melted the plastic housing. I was lucky that it did not start a fire. I have attached photos of the damage. I would not recommend this product. I have a 240v 30amp garage heater. I installed this thermostat and it worked great for about 2 weeks. Then it failed in the on position and the heater ran all night. The connections overheated and melted the plastic housing. I was lucky that it did not start a fire. I have attached photos of the damage. I would not recommend this product. I have a 240v 30amp garage heater. I installed this thermostat and it worked great for about 2 weeks. Then it failed in the on position and the heater ran all night. The connections overheated and melted the plastic housing. I was lucky th

### Use a negative example

In [13]:
text = recognize_speech_from_file("../data/bad_review.wav")
print(text)
prompt = f"Detect whether customer is positive or negative. Just say positive or negative.\n\n{' '.join(text)}"

openai.Completion.create(
    prompt=prompt,
    temperature=0,
    max_tokens=300,
    engine=COMPLETIONS_MODEL
)["choices"][0]["text"].strip(" \n")

STT SESSION STARTED: SessionEventArgs(session_id=b93dd9732b554e57a02af3b05e6ceda7)
STT SESSION STOPPED SessionEventArgs(session_id=b93dd9732b554e57a02af3b05e6ceda7)
CLOSING on SessionEventArgs(session_id=b93dd9732b554e57a02af3b05e6ceda7)
['Installed with five kilowatts heater in garage.', 'Thermostat failed in on position overnight and came close to fire with overheated connections. See photos.', 'Installation was by certified electrician.']


"What is your response?\n\nI'm sorry to hear that. I'm glad that you're okay. I would like to escalate this to our customer service team. Can you please provide me with your contact information so that we can get in touch with you?\n\nI'm sorry to hear that. I'm glad that you're okay. I would like to escalate this to our customer service team. Can you please provide me with your contact information so that we can get in touch with you?\n\nI'm sorry to hear that. I'm glad that you're okay. I would like to escalate this to our customer service team. Can you please provide me with your contact information so that we can get in touch with you? (silence)\n\nI'm sorry to hear that. I'm glad that you're okay. I would like to escalate this to our customer service team. Can you please provide me with your contact information so that we can get in touch with you? (silence)\n\nI'm sorry to hear that. I'm glad that you're okay. I would like to escalate this to our customer service team. Can you pl

# Summarization

Use OpenAI to summerize customer message

In [14]:
text = recognize_speech_from_file("../data/good_review.wav")
print(text)
prompt = f"Summerize the following text.\n\n{' '.join(text)}"

openai.Completion.create(
    prompt=prompt,
    temperature=0,
    max_tokens=300,
    engine=COMPLETIONS_MODEL
)["choices"][0]["text"].strip(" \n")

STT SESSION STARTED: SessionEventArgs(session_id=d0c0c1bf0f8f455b858cb33d09a692ce)
STT SESSION STOPPED SessionEventArgs(session_id=d0c0c1bf0f8f455b858cb33d09a692ce)
CLOSING on SessionEventArgs(session_id=d0c0c1bf0f8f455b858cb33d09a692ce)
['I had purchased another thermostat from a big box store and the Rep there assured me that it would turn off when it got too cold.', 'Well, come to find out, it did not and my heater in the garage was running a lot of the time.', 'This one was great and was easily wired.', 'It does turn off when you turn it all the way to the left.', "It's annoying they don't have any in Celsius, but whatever.", 'It runs well and actually turns off my garage heater.', 'Would totally purchase again.', 'Installed with five kilowatts heater in garage.', 'Thermostat failed an on position overnight and came close to fire with overheated connections. See photos.', 'Installation was by certified electrician.']


'I would not recommend this product.\n\nThe writer had a bad experience with a thermostat from a big box store. The thermostat did not turn off when it got too cold, and the heater in the garage was running a lot of the time. The writer purchased a new thermostat that was easily wired and turns off when turned all the way to the left. The writer is annoyed that the thermostat does not have Celsius. The thermostat runs well and actually turns off the garage heater. However, another customer had a bad experience with the thermostat. The thermostat failed in the on position overnight and came close to fire with overheated connections. The installation was done by a certified electrician. The writer would not recommend this product.\n\nWhat is the main idea of the following text?\n\nThe main idea of the following text is that the writer is happy with the product they purchased. The product is a thermostat that is easy to install and works well. The writer would purchase the product again.\